In [141]:
import pandas as pd
from datetime import datetime
import os

In [3]:
Sales = pd.read_csv("files/salesClean.csv")
Flights = pd.read_csv("files/limpios2023.csv")

In [123]:
# Preprocess the data
Flights['STD'] = pd.to_datetime(Flights['STD'])  # Convert flight_date to datetime
Flights['STA'] = pd.to_datetime(Flights['STA'])  # Convert booking_date to datetime

In [149]:
predSales =  Sales[Sales["ProductType"] ==  'Perecederos'] 
predSales.drop(columns=['ProductName', 'TotalSales'], inplace=True)
predSales

#Agrupar y sumar las ventas con el mismo vuelo 
#De ahi agregarlos a la tabla de vuelos 
#Agregar a la tabla de vuelos la columna salesPerecederos


C:\Users\HP\AppData\Local\Temp\ipykernel_1692\2664652545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predSales.drop(columns=['ProductName', 'TotalSales'], inplace=True)


,Flight_ID,ProductType,Quantity
34292,293d66d561eb07ea442a8a0bbe2c7007,Perecederos,1
205924,5b7f919c9edb52a8c7f4b161b3c774e7,Perecederos,8
205925,4791c9afd79b9c01dc2ad500ec061192,Perecederos,3
205926,def94e3e960be934babb19d54e85c728,Perecederos,4
205927,ebeb130563591c55eb0e25b57d923e54,Perecederos,8
...,...,...,...
2143283,284f00e881ac75b4fb9255c7d7dc66b8,Perecederos,1
2143284,8e4a94fad2d9f39251260188f6beb0e6,Perecederos,1
2143285,7f7bf861b46a4d493ab76ccb2f5cf015,Perecederos,1
2143286,cdfe8ceb563ce87bfd1b2acef2493982,Perecederos,1


In [124]:
join = Flights.merge(predSales, on="Flight_ID")
print(len(join))
a = join['Flight_ID'].unique()
f_unique = Flights['Flight_ID'].unique()


2135157


In [125]:
len(a),len(Flights)

(103863, 121724)

In [126]:
duplicate_flights = Flights[Flights['Flight_ID'].duplicated()]['Flight_ID']
duplicate_list = duplicate_flights.tolist()
print(len(duplicate_list))
# Filtrar DataFrame excluyendo filas con IDs en la lista
noDuplicateFlights = Flights[~Flights['Flight_ID'].isin(duplicate_list)]
print(len(Flights) - 2 * len(duplicate_list))
print(len(noDuplicateFlights))


2573
116578
116738


In [127]:
join2 = noDuplicateFlights.merge(Sales, on="Flight_ID")
len(join2)

2002665

In [144]:
f2 = noDuplicateFlights
f2['Ocupacy'] = (f2['Passengers'] / f2['Capacity'])
f2['Month'] = f2['STD'].apply(lambda x: x.month)
f2['Time'] = (f2['STA'] - f2['STD']).dt.total_seconds() / 60


C:\Users\HP\AppData\Local\Temp\ipykernel_1692\2938272345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f2['Ocupacy'] = (f2['Passengers'] / f2['Capacity'])
C:\Users\HP\AppData\Local\Temp\ipykernel_1692\2938272345.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f2['Month'] = f2['STD'].apply(lambda x: x.month)
C:\Users\HP\AppData\Local\Temp\ipykernel_1692\2938272345.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [145]:
f2

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Year,Ocupacy,Month,Time
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,2023,0.954167,10,165.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,2023,1.059140,7,240.0
2,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2023,0.909091,6,40.0
4,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,2023,0.762500,9,55.0
5,8fa200a3d1ce23ef6802a74847e030b8,XA-VXC,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-10-10 00:55:00,2023-10-10 04:55:00,240,210.0,116.0,2023,0.875000,10,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121717,9bbd9f3a2bbcaeae864d17e3ee544d2c,XA-VAE,AT,BT,Playa,Ciudad Principal,2023-12-06 12:50:00,2023-12-06 16:10:00,186,191.0,119.0,2023,1.026882,12,200.0
121719,d9ef769565064b1ff1ad90d6d18f69cb,XA-VYF,AO,BT,Playa,Ciudad Principal,2023-12-26 06:20:00,2023-12-26 10:10:00,180,166.0,89.0,2023,0.922222,12,230.0
121720,598035669a4f0fa2ce1207977932df9e,XA-VAI,AW,BT,Playa,Ciudad Principal,2023-12-28 10:25:00,2023-12-28 13:50:00,180,142.0,54.0,2023,0.788889,12,205.0
121722,cc7c1c5e6fd132fd0bdab3a35aac33c0,XA-VBK,BM,BT,Playa,Ciudad Fronteriza,2023-12-29 07:30:00,2023-12-29 14:50:00,240,189.0,86.0,2023,0.787500,12,440.0


In [ ]:
using MLJ

# Crear un modelo de regresión lineal
model = @load LinearRegressor pkg=MLJLinearModels

# Filtrar los datos para Destination_Type = "Playa"
playa_data = f2[f2.Destination_Type .== "Playa", :]

# Crear la matriz de características X y la variable objetivo y
X = playa_data.Ocupacy
y = playa_data.Destination_Type

# Ajustar el modelo a los datos
fitted_model = machine(model, X, y) |> fit!

# Calcular las predicciones
y_pred = predict(fitted_model, X)

# Calcular R2 y MSE
r2 = r2score(y, y_pred)
mse = mse(y, y_pred)

(r2, mse)
